In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call

/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


In [2]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Claire_set/"
if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)
    
all_folders_drop, excel_drop, txt_drop = get_dropbox_video_folders("/DATA/TransportROOT/DATA/1_year/", True)
clear_output(wait=False)
all_folders_drop


go
2023-06-27 16:51:11,060-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-27 16:51:11,238-[INFO]- dropbox:474 -> Request to files/list_folder


ApiError: ApiError('0bd0e0b488fb45378536bf55aae56328', ListFolderError('path', LookupError('not_found', None)))

In [ ]:
txt_drop[0][:-13]

In [ ]:
txt_addresses = []
for file in txt_drop:
    file_name = f"{file.split('/')[-3]}{os.sep}{file.split('/')[-2]}{os.sep}{file.split('/')[-1]}"
    if not os.path.exists(directory_targ + file.split('/')[-3] + os.sep+file.split('/')[-2]):
        os.makedirs(directory_targ+ file.split('/')[-3] + os.sep+file.split('/')[-2])
    download(file, directory_targ+file_name)
    txt_addresses.append(directory_targ+file_name)


In [ ]:
series_array = []
# print(txt_addresses)
for address in txt_addresses:
#     print(address)
    series = pd.read_csv(address, sep=": ", engine='python').dropna()
    series_array.append(series.T)
    
dataframe = pd.concat(series_array, axis=0, ignore_index=True)
dataframe["LocalPath"] = txt_addresses

In [ ]:
def month_to_num(x):
    months = {
        'January': '01',
        'February': '02',
        'March': '03',
        'April': '04',
         'May':'05',
         'June':'06',
         'July':'07',
         'August':'08',
         'September':'09',
         'October':'10',
         'November':'11',
         'December':'12'
        }
    a = x.strip()[:3].lower()
    try:
        ez = months[a]
        return ez
    except:
        raise ValueError('Not a month')

def reshape_video_info_df(video_info):
    clean_video_info = video_info.copy()
    clean_video_info["DateTime"] = [date.split(',')[-2] for date in clean_video_info["DateTime"]]
    clean_video_info["DateTime"] = [int(f"{date.split(' ')[-1]}{month_to_num(date.split(' ')[-2])}{date.split(' ')[-3]}") for date in clean_video_info["DateTime"]]
    clean_video_info["Plate"] = [str(entry) for entry in clean_video_info["Plate"]]
    clean_video_info["Time"] = [int(time.split(' ')[-2]) for time in clean_video_info["Time"]]
    clean_video_info["Magnification"] = [entry.split(" ")[-2][:-1] for entry in clean_video_info["Operation"]]
    clean_video_info["BF_or_F"] = [["F", "BF"][entry.split(" ")[-1] == 'Brightfield']for entry in clean_video_info["Operation"]]
#     for i, entry in enumerate(clean_video_info["X"]):
#         print(str(entry.lower()) == "auto")
#         print(entry.split(" "))
#         print(float(entry.split(" ")[14]))
#         print(clean_video_info["DateTime"][i])
#         print(clean_video_info["Run"][i])
    clean_video_info["ExposureTime"] = [entry.split(" ")[3] for entry in clean_video_info["ExposureTime"]]
    clean_video_info["FrameRate"] = [float(entry.split(" ")[-3]) for entry in clean_video_info["FrameRate"]]
    clean_video_info["Binning"] = [int(entry.split("x")[-1]) for entry in clean_video_info["Binning"]]
    clean_video_info["X"] = [float(entry.split(" ")[14]) for entry in clean_video_info["X"]]
    clean_video_info["Y"] = [float(entry.split(" ")[14]) for entry in clean_video_info["Y"]]
    clean_video_info["Z"] = [float(entry.split(" ")[14]) for entry in clean_video_info["Z"]]
    return clean_video_info


new_dataframe = reshape_video_info_df(dataframe)
new_dataframe["StoragePath"] = [address[:-13] + "Img/" for address in txt_drop]
# new_dataframe.to_csv(f"{directory_targ}/PlateInfo.csv")
new_dataframe

## Download section
